In [ ]:
from IPython.display import display, clear_output, Javascript, Markdown
import ipywidgets as ipw
import widgets
import utils
import os
import base64
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

# Dropdown box for getting openBIS objects
publication_selector = widgets.PublicationSelectionWidget()
publication_selector.load_dropdown_box()

prompt_textarea = utils.Textarea(description = "Prompt: ")

download_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Download', icon = 'download', layout = ipw.Layout(width = '100px', height = '50px')
)

quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

download_close_buttons_hbox = ipw.HBox([download_button, quit_button])
increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def download_openbis_object_metadata(b):
    if publication_selector.dropdown.value == -1:
        return
    else:
        openbis_selected_object_permid = publication_selector.dropdown_boxes.children[0].value
        openbis_selected_object = OPENBIS_SESSION.get_sample(openbis_selected_object_permid)

        # Collect all the information about the object from openBIS
        parent_child_relationships = openbis_selected_object.props.all()
        parent_child_relationships["perm_id"] = openbis_selected_object_permid
        parent_child_relationships["type"] = str(openbis_selected_object.type)
        selected_object_schema = utils.get_parent_child_relationships_nested(
            OPENBIS_SESSION, openbis_selected_object, parent_child_relationships
        )
        
        # Export to JSON
        json_data = json.dumps(selected_object_schema, indent=4).encode('utf-8')
        json_data = base64.b64encode(json_data).decode('utf-8')
        
        display(Javascript(f"""
            var a = document.createElement('a');
            a.href = 'data:text/json;base64,{json_data}';
            a.download = 'data.json';
            document.body.appendChild(a);
            a.click();
            document.body.removeChild(a);
        """))

# Export publication metadata

## Select publication

In [ ]:
display(publication_selector)
display(prompt_textarea)
display(download_close_buttons_hbox)
display(increase_buttons_size)
download_button.on_click(download_openbis_object_metadata)
quit_button.on_click(close_notebook)